In [0]:
# --------------------------------------------------------
# 02. Silver | Tratamento, padronização e tipagem (SIDRA 6579)
# --------------------------------------------------------

from pyspark.sql import functions as F

# 1. Fonte Bronze
bronze_tbl = "main.medallion.bronze_sidra_t6579"
silver_tbl = "main.medallion.silver_sidra_t6579"

df = spark.table(bronze_tbl)

# 2. Padronização dos nomes de colunas (snake_case simples)
def to_snake(col_name: str) -> str:
    return col_name.strip().lower().replace(" ", "_")

rename_map = {c: to_snake(c) for c in df.columns}
for old, new in rename_map.items():
    df = df.withColumnRenamed(old, new)

# 3. Removção de linha de cabeçalho embutida (SIDRA)
#    Mantém apenas linhas onde d1c é numérico (ex.: 11, 12, 13...)
if "d1c" in df.columns:
    df = df.filter(F.col("d1c").rlike(r"^\d+$"))

# 4. Tipagens com try_cast (evita quebrar caso apareça algo malformado)

int_cols = ["d1c", "d2c", "d3c", "d4c", "d5c", "nc", "mc"]
for c in int_cols:
    if c in df.columns:
        df = df.withColumn(c, F.expr(f"try_cast({c} as int)"))

# 5. Valor (v) costuma vir como string; normaliza vírgula para ponto
if "v" in df.columns:
    df = df.withColumn(
        "v",
        F.expr("try_cast(regexp_replace(cast(v as string), ',', '.') as double)")
    )

# 6. Regras básicas de qualidade
key_cols = [c for c in ["d1c", "d2c", "d3c"] if c in df.columns]
for c in key_cols:
    df = df.filter(F.col(c).isNotNull())

if "v" in df.columns:
    df = df.filter(F.col("v").isNotNull())

# 7. Seleção final
cols = df.columns
preferred_order = [
    "d1c","d1n","d2c","d2n","d3c","d3n","d4c","d4n","d5c","d5n",
    "v","mn","mc","nc",
    "ingestion_ts","source","source_url"
]
final_cols = [c for c in preferred_order if c in cols] + [c for c in cols if c not in preferred_order]

df_silver = df.select(*final_cols)

# 8. Gravação Silver (reprocessável)
spark.sql("DROP TABLE IF EXISTS main.medallion.silver_sidra_t6579")

(
    df_silver
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(silver_tbl)
)

# 9. Evidências
print("Silver gravado com sucesso:", silver_tbl)
display(df_silver.limit(10))
print("Total de linhas (Silver):", df_silver.count())
